In [1]:
import os
import pyodbc
import pandas as pd

In [2]:
os.listdir('./../../repos/LASOS-INT/TTBsim/01_Config/DBFiles/')

['AGFA.mdb',
 'Modelo de Dados - ToolBoxMix.png',
 'off',
 'off2',
 'ToolBoxMix_Development - Faz Tudo Quero Logo.ldb',
 'ToolBoxMix_Development - Faz Tudo Quero Logo.mdb',
 'ToolBoxMix_Development.ldb',
 'ToolBoxMix_Development.mdb',
 'ToolBoxMix_InjecaoPlastica.mdb',
 'ToolBoxMix_MetalMeca - backup.mdb',
 'ToolBoxMix_MetalMeca.mdb']

In [3]:
path = './../../repos/LASOS-INT/TTBsim/01_Config/DBFiles/ToolBoxMix_Development - Faz Tudo Quero Logo.mdb'
conn_str =     conn_str = (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};' +
            'DBQ='+f'{path};')
print(conn_str)
conn = pyodbc.connect(conn_str)

DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=./../../repos/LASOS-INT/TTBsim/01_Config/DBFiles/ToolBoxMix_Development - Faz Tudo Quero Logo.mdb;


In [4]:
cursor = conn.cursor()
for i in cursor.tables(tableType='TABLE'):
    print(i.table_name)

ACABADO
AGRU_ENTID
AGRU_GRUPO
AGRU_LINK
ALTERNATIVAS
ATIV_CENTRO
ATIV_PRECED
ATIVP
CAL_EXCE
CAL_PART
COMPRADO
CONJUGADAS
CONJUNTOS
CTRL_APONTA
CTRL_ESTOQ_DEPO
CTRL_ESTOQ_MOV
CTRL_ESTOQ_POS
CTRL_MARCO
DEC_CRITERIOS_DE_SEQUENCIAMENTO
DEC_DEDICACAO_MAQUINAS
DEC_FLEXIBILIDADES
DEC_HORA_EXTRA
DEC_HORARIOS
DEC_LOTES_PRODUCAO
DEC_LOTES_TRANSFERENCIA
DEC_PLANO_COMPRAS
DEC_PRAZOS_ENTREGAS
DEC_PRIORIDADE
DEC_RESTRICAO_INICIO
DEC_SIMULAR_ORDEM_PRODUCAO
DEC_SPLITS
DEC_SUBCONTRATACOES
DEC_TEMPO_TRANS
ELO_PERDIDO
Erros ao colar
ESCALARES
ESTOQUES
ESTOQUES_PROCESSO
EXT_COMPRA
EXT_COMPRA_RECEB
EXT_EMPRESA
EXT_PLANO_MESTRE
EXT_VENDA
EXT_VENDA_ENTREGA
EXT_VENDA_STATUS
FABRICA
FABRICA_OLD
FERRAMENTA
FLEX_ESCOPO_MAQ
FLEX_ESCOPO_OPER
HORARIOS
HORARIOS_PARTICAO
ITEM
ITEM_ESTRU
ITENS_NAVEGACAO
ITENS_SELECAO
LISTA_APLICACOES
LISTA_APLICACOES_AUX
MAQUINA
OPERACOES
REL_ANALISE_FINANCEIRA
REL_DEBUG
REL_DEMANDAS_LIQUIDAS
REL_ENTREGAS_NAO_REALIZADAS
REL_NECESS_MAT
REL_PONTUALIDADE
REL_RECEB_MAT
REL_SEQUENCIA_DETAL

In [5]:
ENTREGAS = pd.read_sql('select * from EXT_VENDA_ENTREGA', conn)
ENTREGAS.sort_values(by='CdItem', inplace=True)
ENTREGAS

C:\Users\arthur.silva\AppData\Local\Temp\ipykernel_13868\1964889840.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ENTREGAS = pd.read_sql('select * from EXT_VENDA_ENTREGA', conn)


,CdVenda,CdEntrega,DsEntrega,CdTpElo,CdItem,CdAtiv,DtEntrega,HrEntrega,QtEntrega,VlUnit,VlMultaDia,FgNaoSimu
0,Venda1,EntregaA,None,ITEM,A,-1,2022-04-14,-1,1.0,6360.0,500.0,0
1,Venda1,EntregaB,,ITEM,B,-1,2022-04-15,-1,1.0,5900.0,650.0,0
2,Venda1,EntregaC,,ITEM,C,-1,2022-04-16,-1,1.0,7840.0,800.0,0
3,Venda1,EntregaD,,ITEM,D,-1,2022-04-17,-1,1.0,7860.0,950.0,0
4,Venda1,EntregaE,,ITEM,E,-1,2022-04-15,-1,1.0,8380.0,2000.0,0


In [131]:
TAREFAS = pd.read_sql('select * from REL_TAREFAS', conn)
TAREFAS.sort_values(by=['CdAtiv', 'NuEstagio', 'CdMaq'], inplace=True)
TAREFAS.head()

C:\Users\arthur.silva\AppData\Local\Temp\ipykernel_13868\2233178274.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  TAREFAS = pd.read_sql('select * from REL_TAREFAS', conn)


,IdTar,IdTarAgreg,CdMaq,CdFerr,CdEntrega,CdAcabado,DtEntrega,CdAtiv,CdItem,NuEstagio,...,TmRepouso,TmTrans,TmLiqSetup,TmLiqProc,TmLiqRepouso,TmLiqTrans,QtProg,NumCriterio,FgAtrasou,FgEntrega
0,0,0,TRACAR,-,EntregaA,None,None,A#1,None,None,...,0.0,86400.0,0.0,86400.0,0.0,86400.0,1.0,-1,False,False
2,2,2,TORNEAR,-,EntregaA,None,None,A#2,None,None,...,0.0,86400.0,0.0,172800.0,0.0,86400.0,1.0,-1,False,False
7,7,7,MANDRILAR,-,EntregaA,None,None,A#3,None,None,...,0.0,86400.0,0.0,172800.0,0.0,86400.0,1.0,-1,False,False
12,12,12,FRESAR,-,EntregaA,None,None,A#4,None,None,...,0.0,86400.0,0.0,86400.0,0.0,86400.0,1.0,-1,False,False
16,16,16,FURAR,-,EntregaA,None,None,A#5,None,None,...,0.0,86400.0,0.0,86400.0,0.0,86400.0,1.0,-1,False,True


In [137]:
for tar in TAREFAS.itertuples():
    print(tar.IniProc.date(), tar.CdMaq)

2022-04-01 TRACAR
2022-04-03 TORNEAR
2022-04-06 MANDRILAR
2022-04-09 FRESAR
2022-04-11 FURAR
2022-04-03 TRACAR
2022-04-05 FRESAR
2022-04-08 MANDRILAR
2022-04-10 FURAR
2022-04-14 TORNEAR
2022-04-06 TRACAR
2022-04-08 FURAR
2022-04-11 TORNEAR
2022-04-15 MANDRILAR
2022-04-18 FRESAR
2022-04-02 TRACAR
2022-04-04 MANDRILAR
2022-04-07 TORNEAR
2022-04-10 FRESAR
2022-04-04 TRACAR
2022-04-09 MANDRILAR
2022-04-12 FRESAR
2022-04-15 TORNEAR
2022-04-17 FRESAR


In [83]:
CALENDARIO = pd.read_sql('select * from HORARIOS_PARTICAO', conn)
CALENDARIO

C:\Users\arthur.silva\AppData\Local\Temp\ipykernel_13868\62351689.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  CALENDARIO = pd.read_sql('select * from HORARIOS_PARTICAO', conn)


,HorarioCod,DiaDaSemana,DiaDaSemanaExibir,Turno1,InicioTurno1,FimTurno1,TipoTurno1,Turno2,InicioTurno2,FimTurno2,TipoTurno2,Turno3,InicioTurno3,FimTurno3,TipoTurno3,Turno4,InicioTurno4,FimTurno4,TipoTurno4
0,HORÁRIO 1,1,1,1,1899-12-30 00:00:00,1899-12-30 12:00:00,Normal,2,1899-12-30 12:00:00,1899-12-30 16:00:00,Normal,3,1899-12-30 16:00:00,1899-12-30 20:00:00,Normal,4,1899-12-30 20:00:00,1899-12-30 00:00:00,Normal
1,HORÁRIO 1,2,2,1,1899-12-30 00:00:00,1899-12-30 12:00:00,Normal,2,1899-12-30 12:00:00,1899-12-30 16:00:00,Normal,3,1899-12-30 16:00:00,1899-12-30 20:00:00,Normal,4,1899-12-30 20:00:00,1899-12-30 00:00:00,Normal
2,HORÁRIO 1,3,3,1,1899-12-30 00:00:00,1899-12-30 12:00:00,Normal,2,1899-12-30 12:00:00,1899-12-30 16:00:00,Normal,3,1899-12-30 16:00:00,1899-12-30 20:00:00,Normal,4,1899-12-30 20:00:00,1899-12-30 00:00:00,Normal
3,HORÁRIO 1,4,4,1,1899-12-30 00:00:00,1899-12-30 12:00:00,Normal,2,1899-12-30 12:00:00,1899-12-30 16:00:00,Normal,3,1899-12-30 16:00:00,1899-12-30 20:00:00,Normal,4,1899-12-30 20:00:00,1899-12-30 00:00:00,Normal
4,HORÁRIO 1,5,5,1,1899-12-30 00:00:00,1899-12-30 12:00:00,Normal,2,1899-12-30 12:00:00,1899-12-30 16:00:00,Normal,3,1899-12-30 16:00:00,1899-12-30 20:00:00,Normal,4,1899-12-30 20:00:00,1899-12-30 00:00:00,Normal
5,HORÁRIO 1,6,6,1,1899-12-30 00:00:00,1899-12-30 12:00:00,Normal,2,1899-12-30 12:00:00,1899-12-30 16:00:00,Normal,3,1899-12-30 16:00:00,1899-12-30 20:00:00,Normal,4,1899-12-30 20:00:00,1899-12-30 00:00:00,Normal
6,HORÁRIO 1,7,7,1,1899-12-30 00:00:00,1899-12-30 12:00:00,Normal,2,1899-12-30 12:00:00,1899-12-30 16:00:00,Normal,3,1899-12-30 16:00:00,1899-12-30 20:00:00,Normal,4,1899-12-30 20:00:00,1899-12-30 00:00:00,Normal
7,HORÁRIO 2,1,1,1,1899-12-30 00:01:00,1899-12-30 08:00:00,Normal,2,1899-12-30 08:00:00,1899-12-30 16:00:00,Normal,3,1899-12-30 16:00:00,1899-12-30 20:00:00,Normal,4,1899-12-30 20:00:00,1899-12-30 00:01:00,Normal
8,HORÁRIO 2,2,2,1,1899-12-30 00:01:00,1899-12-30 08:00:00,Normal,2,1899-12-30 08:00:00,1899-12-30 16:00:00,Normal,3,1899-12-30 16:00:00,1899-12-30 20:00:00,Normal,4,1899-12-30 20:00:00,1899-12-30 00:01:00,Normal
9,HORÁRIO 2,3,3,1,1899-12-30 00:01:00,1899-12-30 08:00:00,Normal,2,1899-12-30 08:00:00,1899-12-30 16:00:00,Normal,3,1899-12-30 16:00:00,1899-12-30 20:00:00,Normal,4,1899-12-30 20:00:00,1899-12-30 00:01:00,Normal


In [124]:
initial_date = pd.read_sql('select DtHrIniSim from TAB_HORIZONTE', conn).iloc[0].values[0]
import datetime
initial_date = pd.to_datetime(initial_date)
day = np.datetime64(initial_date.date()).astype(datetime.datetime).isoweekday()
day, initial_date

C:\Users\arthur.silva\AppData\Local\Temp\ipykernel_13868\608489023.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  initial_date = pd.read_sql('select DtHrIniSim from TAB_HORIZONTE', conn).iloc[0].values[0]


(5, Timestamp('2022-04-01 00:00:01'))

In [126]:
t = '2023-04-24'
t = np.datetime64(t)
day = t.astype(datetime.datetime).isoweekday()
t, day, t.astype(datetime.datetime)

(numpy.datetime64('2023-04-24'), 1, datetime.date(2023, 4, 24))